# Recurrent Neural Network

This notebook will read in the merged dataset of session behavior, track IDs and track information, preprocess the data and perform an RNN model using an LSTM layer. 

The RNN model was chosen here because the sequence of the rows is important. The dataset is composed of several unique session IDs, each session consists of 20 rows. The data is reshaped during preprocessing to group the sessions. The LSTM layer is capable of learning long term dependencies, but predicting the following sequence based on the recent instances. 

### Import libraries and read in the dataset

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import models
from keras import layers
from keras import optimizers
from sklearn import metrics

#from src folder
from preprocessing import *

In [2]:
df = pd.read_csv('tracks_session_clean.csv', index_col=0)

##There is a variety of session lengths. The sessions with 20 instances will be used.

df = df.loc[df['session_length'] == 20]

### Drop columns

To reduce making assumptions about the metadata, the acoustic vector columns were dropped. There are still plenty of music features available in the dataset that are more interpretable. Other dropped columns contained multicolinearity from the data exploration.

In [3]:
drop_col = ['session_length', 'date', 'track_id', 'hist_user_behavior_reason_start', 'beat_strength', 'dyn_range_mean',
'acoustic_vector_0', 'acoustic_vector_1', 'acoustic_vector_2', 'acoustic_vector_3', 'acoustic_vector_4', 'acoustic_vector_5', 'acoustic_vector_6', 'acoustic_vector_7']

df = df.drop(labels=drop_col, axis=1).reset_index(drop=True)

### Preprocessing

Takes in a dataframe. Returns `train_features`, `test_features`, `train_labels`, and `test_labels`. It also scales the features, then reshapes them to fit the 3D LSTM layer. In this case, the 3D layer follows this format 
  (`num samples, num time-steps, num features`)

In [4]:
train_features, test_features, train_labels, test_labels = preprocess(df)

In [5]:
print(train_features.shape)
print(train_labels.shape)
print(test_features.shape)
print(test_labels.shape)

(3500, 15, 32)
(3500, 15, 1)
(1566, 15, 32)
(1566, 15, 1)


### Build & train model

In [6]:
model = keras.Sequential()
model.add(layers.Dense(32, activation='tanh'))
model.add(layers.LSTM(16, activation='relu', return_sequences=True))
model.add(layers.Dropout(.2))
model.add(layers.Dense(16, activation='tanh'))
model.add(layers.Dense(8, activation='relu'))

#Binary outcome
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()], optimizer='Adam')

In [7]:
model.fit(train_features, train_labels, epochs=200, verbose=2, shuffle=False)

Epoch 1/200
110/110 - 3s - loss: 0.6848 - accuracy: 0.5553 - precision: 0.5629 - recall: 0.8384 - 3s/epoch - 31ms/step
Epoch 2/200
110/110 - 1s - loss: 0.6779 - accuracy: 0.5712 - precision: 0.5815 - recall: 0.7729 - 597ms/epoch - 5ms/step
Epoch 3/200
110/110 - 1s - loss: 0.6735 - accuracy: 0.5843 - precision: 0.5947 - recall: 0.7556 - 626ms/epoch - 6ms/step
Epoch 4/200
110/110 - 1s - loss: 0.6702 - accuracy: 0.5895 - precision: 0.5993 - recall: 0.7541 - 614ms/epoch - 6ms/step
Epoch 5/200
110/110 - 1s - loss: 0.6682 - accuracy: 0.5931 - precision: 0.6025 - recall: 0.7539 - 614ms/epoch - 6ms/step
Epoch 6/200
110/110 - 1s - loss: 0.6667 - accuracy: 0.5948 - precision: 0.6041 - recall: 0.7533 - 663ms/epoch - 6ms/step
Epoch 7/200
110/110 - 1s - loss: 0.6658 - accuracy: 0.5954 - precision: 0.6046 - recall: 0.7536 - 631ms/epoch - 6ms/step
Epoch 8/200
110/110 - 1s - loss: 0.6646 - accuracy: 0.5977 - precision: 0.6074 - recall: 0.7493 - 642ms/epoch - 6ms/step
Epoch 9/200
110/110 - 1s - loss: 0

In [8]:
loss, accuracy, precision, recall = model.evaluate(test_features, test_labels)

49/49 [==============================] - 0s 3ms/step - loss: 0.7712 - accuracy: 0.5603 - precision: 0.5790 - recall: 0.6994


In [9]:
print(accuracy)
print(precision)
print(recall)
print((2*precision*recall)/(precision+recall)) #F1 score

0.5603235363960266
0.5790258646011353
0.69941246509552
0.6335509471322516


### Interpretation

The model learned the sequential nature of the dataset, which the logistic regression was unable to do. The precision score was 72% and 58% on train and test, respectively. We can consider this model slightly overfit, and may perform better once exposed to the original dataset and its noise. This model also included the musical features, which the other classifiers did not. 